In [2]:
import pandas as pd

df = pd.read_csv("./data.csv")

In [19]:
print(df.head())
print(df.describe())
for column in df.columns:
    print(f"-> Column: {column}")
    print(df[column].value_counts())
    print("\n------\n")

                                   name  money new&used  brand  Year  \
0               2023 Acura Integra Base  33095      New  Acura  2023   
1                 2023 Acura Mdx A-Spec  61745      New  Acura  2023   
2  2023 Acura Integra A-Spec Technology  38095      New  Acura  2023   
3                 2023 Acura Tlx A-Spec  47995      New  Acura  2023   
4                 2023 Acura Tlx A-Spec  47995      New  Acura  2023   

                       Model          Exterior color  Mileage Fuel type  
0               Integra Base  Liquid Carbon Metallic        5  Gasoline  
1                 Mdx A-Spec    Majestic Black Pearl        0  Gasoline  
2  Integra A-Spec Technology    Majestic Black Pearl        5  Gasoline  
3                 Tlx A-Spec    Majestic Black Pearl       11  Gasoline  
4                 Tlx A-Spec    Platinum White Pearl        2  Gasoline  
              money           Year        Mileage
count  1.722160e+05  172216.000000  172216.000000
mean   3.437182e+04    

In [3]:
df_cluster_all = df.groupby(["name","new&used","brand","Year","Model"], as_index=False).agg(
    num=("name", "count"),
    money_avg=("money", "mean"),
    mileage_avg=("Mileage", "mean")
)
df_cluster_all["money_avg"] = df_cluster_all["money_avg"].round(2)
df_cluster_all["mileage_avg"] = df_cluster_all["mileage_avg"].round(2)
df_cluster_all


,name,new&used,brand,Year,Model,num,money_avg,mileage_avg
0,2012 Acura Mdx 3.7L,Used,Acura,2012,Mdx 3.7L,18,14631.28,116020.50
1,2012 Acura Mdx 3.7L Advance,Used,Acura,2012,Mdx 3.7L Advance,8,16033.38,136720.25
2,2012 Acura Mdx 3.7L Technology,Used,Acura,2012,Mdx 3.7L Technology,23,15176.74,116289.04
3,2012 Acura Rdx Base,Used,Acura,2012,Rdx Base,1,12699.00,117671.00
4,2012 Acura Rdx Technology,Used,Acura,2012,Rdx Technology,4,13325.00,100579.25
...,...,...,...,...,...,...,...,...
19526,2023 Volvo Xc90 Recharge Plug-In Hybrid Ultima...,New,Volvo,2023,Xc90 Recharge Plug-In Hybrid Ultimate Bright T...,2,79723.50,2.50
19527,2023 Volvo Xc90 Recharge Plug-In Hybrid Ultima...,Used,Volvo,2023,Xc90 Recharge Plug-In Hybrid Ultimate Bright T...,1,84995.00,1225.00
19528,2023 Volvo Xc90 Recharge Plug-In Hybrid Ultima...,New,Volvo,2023,Xc90 Recharge Plug-In Hybrid Ultimate Dark Theme,1,81330.00,0.00
19529,2023 Volvo Xc90 Ultimate,New,Volvo,2023,Xc90 Ultimate,3,69106.67,0.00


In [4]:
df_cluster_all.to_csv("./data_cluster.csv", index=False)

In [5]:
df_cluster_name = df.groupby(["name"], as_index=False).agg(
    num=("name", "count"),
    money_avg=("money", "mean"),
    mileage_avg=("Mileage", "mean"),
    year=("Year",lambda x: list(set(x))),
    brand=("brand",lambda x: list(set(x))),
    model=("Model",lambda x: list(set(x))),
    new_and_used=("new&used",lambda x: list(set(x)))
)
df_cluster_name["money_avg"] = df_cluster_name["money_avg"].round(2)
df_cluster_name["mileage_avg"] = df_cluster_name["mileage_avg"].round(2)
df_cluster_name

,name,num,money_avg,mileage_avg,year,brand,model,new_and_used
0,2012 Acura Mdx 3.7L,18,14631.28,116020.50,[2012],[Acura],[Mdx 3.7L],[Used]
1,2012 Acura Mdx 3.7L Advance,8,16033.38,136720.25,[2012],[Acura],[Mdx 3.7L Advance],[Used]
2,2012 Acura Mdx 3.7L Technology,23,15176.74,116289.04,[2012],[Acura],[Mdx 3.7L Technology],[Used]
3,2012 Acura Rdx Base,1,12699.00,117671.00,[2012],[Acura],[Rdx Base],[Used]
4,2012 Acura Rdx Technology,4,13325.00,100579.25,[2012],[Acura],[Rdx Technology],[Used]
...,...,...,...,...,...,...,...,...
15506,2023 Volvo Xc90 Recharge Plug-In Hybrid Ultimate,1,80571.00,10.00,[2023],[Volvo],[Xc90 Recharge Plug-In Hybrid Ultimate],[New]
15507,2023 Volvo Xc90 Recharge Plug-In Hybrid Ultima...,3,81480.67,410.00,[2023],[Volvo],[Xc90 Recharge Plug-In Hybrid Ultimate Bright ...,"[New, Used]"
15508,2023 Volvo Xc90 Recharge Plug-In Hybrid Ultima...,1,81330.00,0.00,[2023],[Volvo],[Xc90 Recharge Plug-In Hybrid Ultimate Dark Th...,[New]
15509,2023 Volvo Xc90 Ultimate,3,69106.67,0.00,[2023],[Volvo],[Xc90 Ultimate],[New]


In [6]:
df_cluster_name.to_csv("./data_cluster_name.csv", index=False)

In [7]:
df_cluster_name = df.groupby(["Year"], as_index=False).agg(
    name=("name", lambda x: list(set(x))),
    money_avg=("money", "mean"),
    mileage_avg=("Mileage", "mean"),
    brand=("brand",lambda x: list(set(x))),
    model=("Model",lambda x: list(set(x))),
    new_and_used=("new&used",lambda x: list(set(x)))
)
df_cluster_name["money_avg"] = df_cluster_name["money_avg"].round(2)
df_cluster_name["mileage_avg"] = df_cluster_name["mileage_avg"].round(2)
df_cluster_name

,Year,name,money_avg,mileage_avg,brand,model,new_and_used
0,2012,"[2012 Subaru Impreza 2.0I Premium, 2012 Acura ...",15074.95,112651.23,"[Mitsubishi, Lexus, Mercedes_Benz, Land_Rover,...","[Sonic 1Lt, Rdx Base, 328 I, 335 I, Tl Technol...","[New, Dodge Certified, Used]"
1,2013,"[2013 Subaru Impreza Wrx Base, 2013 Ford Exped...",16229.48,104084.95,"[Mitsubishi, Lexus, Mercedes_Benz, Land_Rover,...","[Avalon Hybrid Xle Premium, Rdx Base, 328 I, 3...","[New, Ram Certified, Porsche Certified, Nissan..."
2,2014,"[2014 Volkswagen Jetta Tdi, 2014 Audi Tts 2.0T...",18464.06,98152.77,"[Mitsubishi, Lexus, Mercedes_Benz, Land_Rover,...","[Avalon Hybrid Xle Premium, Verano Leather Gro...","[Chrysler Certified, New, Ford Certified, Infi..."
3,2015,"[2015 Mazda Cx-9 Grand Touring, 2015 Porsche P...",21287.43,90912.31,"[Mitsubishi, Lexus, Mercedes_Benz, Land_Rover,...","[Verano Leather Group, Rdx Base, Cts 3.6L Perf...","[Chrysler Certified, New, Ford Certified, Infi..."
4,2016,"[2016 Nissan Frontier Pro-4X, 2016 Chevrolet S...",23467.49,81599.62,"[Mitsubishi, Lexus, Mercedes_Benz, Land_Rover,...","[328 I, Q50 3.0T Premium, Corolla S Plus, Mkc ...","[Chrysler Certified, New, Ford Certified, Infi..."
5,2017,"[2017 Bmw 440 Gran Coupe I, 2017 Porsche 911 C...",25973.90,72954.98,"[Mitsubishi, Lexus, Mercedes_Benz, Land_Rover,...","[Q50 3.0T Premium, Mkc Select, Mustang V6, Lan...","[Chrysler Certified, New, Ford Certified, Infi..."
6,2018,"[2018 Subaru Wrx Sti Limited W/ Lip, 2018 Pors...",29612.19,61023.69,"[Mitsubishi, Lexus, Mercedes_Benz, Land_Rover,...","[Macan, Mkc Select, Sq5 3.0T Prestige, Is 350 ...","[Chrysler Certified, New, Ford Certified, Gmc ..."
7,2019,"[2019 Mitsubishi Eclipse Cross Se, 2019 Volksw...",33217.17,45875.04,"[Mitsubishi, Lexus, Mercedes_Benz, Land_Rover,...","[Macan, Ux 200 Ux 200, Mkc Select, Sedona L, P...","[Chrysler Certified, New, Ford Certified, Gmc ..."
8,2020,"[2020 Audi A7 55 Prestige, 2020 Bmw X4 M Awd, ...",36159.57,39016.61,"[Mitsubishi, Lexus, Mercedes_Benz, Land_Rover,...","[Macan, Is 350 Is 350 F Sport, Sq8 4.0T Quattr...","[Chrysler Certified, New, Ford Certified, Gmc ..."
9,2021,"[2021 Jeep Wrangler Rubicon, 2021 Mercedes-Ben...",41900.78,29287.85,"[Mitsubishi, Lexus, Mercedes_Benz, Land_Rover,...","[Macan, Ux 200 Ux 200, Canyon Elevation Standa...","[Chrysler Certified, New, Ford Certified, Gmc ..."


In [4]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Tạo lưới biểu đồ chia theo cột 'new&used'
g = sns.FacetGrid(df, col='new&used', height=5, aspect=1.2)

# Vẽ scatter plot cho từng nhóm
g.map_dataframe(sns.scatterplot, x='name', y='money', s=100, color='b')

# Tùy chỉnh tiêu đề
g.set_axis_labels("Tên sản phẩm", "Giá trị (money)")
g.set_titles(col_template="{col_name}")

plt.show()


KeyboardInterrupt: 

In [3]:
df_rec = df[df['new&used'].isin(['New', 'Used'])]
df_rec = df_rec[df_rec['Fuel type'].isin(['Gasoline'])]
df_rec.to_csv("./data_rec.csv", index=False)

In [4]:
df_rec
print(df_rec.count(
    
))


name              145645
money             145645
new&used          145645
brand             145645
Year              145645
Model             145645
Exterior color    145645
Mileage           145645
Fuel type         145645
dtype: int64


In [4]:
!pip install scikit-learn

In [14]:
import numpy as np
from sklearn.linear_model import SGDRegressor
from sklearn.model_selection import train_test_split

train_data, test_data = train_test_split(df_rec, test_size=0.2, random_state=42)
X_train = train_data[['Year', 'Mileage']]
X_test = test_data[['Year', 'Mileage']]
y_train = train_data[['money']].values
sgd = SGDRegressor(max_iter=1000, tol=1e-3, learning_rate='constant', eta0=0.1)
sgd.fit(X_train, y_train.ravel())
y_pred = sgd.predict(X_test)

print("Dự đoán cho X =", X_test)
print("Giá trị dự đoán y =", y_pred)

Dự đoán cho X =         Year  Mileage
10065   2023       11
155249  2023        6
167335  2018    27999
117417  2022     4000
11509   2021     9164
...      ...      ...
166100  2020    21415
26451   2018    70041
25202   2020    57701
30357   2020    33995
15112   2017    97654

[29129 rows x 2 columns]
Giá trị dự đoán y = [-1.83092449e+18 -1.82112662e+18 -5.66710242e+19 ... -1.14876095e+20
 -6.84224219e+19 -1.93164293e+20]


In [5]:
pip install catboost

Note: you may need to restart the kernel to use updated packages.


In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from catboost import CatBoostRegressor, Pool

df = pd.read_csv("./data.csv")
# Preload dataset and remove duplicates
# Assuming the data comes from Power BI as columns and the data values are passed into this dataframe
# df = pd.DataFrame({
#     'money': money,
#     'car_name': car_name,
#     'brand': brand,
#     'Exterior_color': Exterior_color,
#     'Fuel_type': Fuel_type,
#     'Mileage': Mileage,
#     'Model': Model,
#     'Year': Year,
#     'new_used': new_used
# })

# Remove duplicates
# dataset = dataset.drop_duplicates()

# Step 2: Prepare the data
# Use 'Mileage' and 'Year' as independent variables (X)
# Use 'money' as the dependent variable (y)
df
df = df[df['new&used'].isin(['New', 'Used'])]
df = df[df['Fuel type'].isin(['Gasoline'])]

# Kiểm tra số lượng dòng sau khi lọc
print(f"Number of rows after filtering: {len(df)}")
if len(df) == 0:
    print("No data left after filtering, check the filtering conditions!")
else:
    df['new&used'] = df['new&used'].replace({'New': 1, 'Used': 0})
    cat_columns = ['name', 'brand', 'Exterior color', 'Model', 'new&used']

    # Chuyển đổi biến phân loại sang kiểu category
    for col in cat_columns:
        df[col] = df[col].astype("category")

    # Tiến hành phân chia dữ liệu
    train_data, test_data = train_test_split(df, test_size=0.2, random_state=42)

    x = ['name', 'brand', 'Exterior color', 'Model', 'Year', 'Mileage', 'new&used']
    y = 'money'
    x_train = df[x]
    y_train = df[y]
    
    x_test = pd.DataFrame({
        'name': ['2023 Acura Integra Base'],
        'brand': ['Acura'],
        'Exterior color': ['Platinum White Pearl'],
        'Model': ['Integra Base'],
        'Year': [2024],
        'Mileage': [0],
        'new&used': ['New']
    })
    # Tạo Pool dữ liệu train và test
    train_pool = Pool(data=x_train, label=y_train, cat_features=cat_columns)
    test_pool = Pool(data=x_test, cat_features=cat_columns)

    # Định nghĩa mô hình và tham số
    model = CatBoostRegressor(
        loss_function='RMSE',
        iterations=300,
        depth=4,
        learning_rate=0.2,
        l2_leaf_reg=1,
        boosting_type='Ordered',
        eval_metric='RMSE',
        verbose=50,
        early_stopping_rounds=50
    )

    # Huấn luyện mô hình
    model.fit(train_pool)

    # Dự đoán trên tập test
    preds = model.predict(test_pool)
    print(f"Predicted value: {preds[0]}")
    # # Tính toán các chỉ số đánh giá
    # mse_value = mean_squared_error(y_test, preds)
    # rmse_value = np.sqrt(mse_value)
    # mae_value = mean_absolute_error(y_test, preds)
    # r2_value = r2_score(y_test, preds)
    # mape = mean_absolute_percentage_error(y_test, preds) * 100

    # # In kết quả
    # print(f"MSE: {mse_value}")
    # print(f"MAE: {mae_value}")
    # print(f"R²: {r2_value}")
    # print(f"MAPE: {mape:.2f}%")
    # print(f"RMSE: {rmse_value}")

    # plt.figure(figsize=(10, 10))
    # plt.scatter(y_test, preds, alpha=0.7, color='blue')
    # plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], '--r', linewidth=2)  # Đường y=x
    # plt.xlabel("Giá trị thực tế (money)")
    # plt.ylabel("Giá trị dự đoán (money)")
    # plt.title("So sánh Giá trị Thực tế vs. Dự đoán")
    # plt.show()


Number of rows after filtering: 145645


C:\Users\LENOVO\AppData\Local\Temp\ipykernel_16356\1666154237.py:39: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['new&used'] = df['new&used'].replace({'New': 1, 'Used': 0})


0:	learn: 20352.9501813	total: 99.2ms	remaining: 29.7s
50:	learn: 12342.2344255	total: 6.47s	remaining: 31.6s
100:	learn: 12214.2514306	total: 11.1s	remaining: 21.9s
150:	learn: 12125.5727296	total: 15.6s	remaining: 15.3s
200:	learn: 11962.1021537	total: 21.3s	remaining: 10.5s
250:	learn: 11875.7728630	total: 26.1s	remaining: 5.1s
299:	learn: 11649.2720515	total: 31.3s	remaining: 0us
Predicted value: 60238.0510792421


In [29]:
!pip install --upgrade pip setuptools wheel
!pip install pyspark

  Using cached setuptools-78.1.0-py3-none-any.whl.metadata (6.6 kB)
Using cached setuptools-78.1.0-py3-none-any.whl (1.3 MB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 58.1.0
    Uninstalling setuptools-58.1.0:
      Successfully uninstalled setuptools-58.1.0
  Using cached pyspark-3.5.5.tar.gz (317.2 MB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Using cached py4j-0.10.9.7-py2.py3-none-any.whl.metadata (1.5 kB)
Using cached py4j-0.10.9.7-py2.py3-none-any.whl (200 kB)
  Created wheel for pyspark: filename=pyspark-3.5.5-py2.py3-none-any.whl size=317747963 sha256=945b9af33efa2bcbf0e0d2be3bd0b803be70d53fbd2854a41146ee1f5aee258f
  Stored in directory: c:\users\lenovo\appdata\local\pip\cache\wheels\af\de\65\358d44d6c11fd5795dcbda700ff8b7ab3cbcdfde7f4fb1d9fb
Successfully built pyspark


In [31]:
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml.feature import VectorAssembler
from pyspark.sql import SparkSession

# Khởi tạo Spark session
spark = SparkSession.builder.appName("BigDataPrediction").getOrCreate()

# Đọc dữ liệu vào DataFrame
df = spark.read.csv("./data.csv", header=True, inferSchema=True)

# Tạo vector features
assembler = VectorAssembler(inputCols=["feature1", "feature2"], outputCol="features")
df = assembler.transform(df)

# Chạy mô hình RandomForest
rf = RandomForestRegressor(featuresCol="features", labelCol="label")
model = rf.fit(df)
predictions = model.transform(df)
predictions.show()

Py4JJavaError: An error occurred while calling None.org.apache.spark.api.java.JavaSparkContext.
: java.lang.UnsupportedOperationException: getSubject is not supported
	at java.base/javax.security.auth.Subject.getSubject(Subject.java:277)
	at org.apache.hadoop.security.UserGroupInformation.getCurrentUser(UserGroupInformation.java:577)
	at org.apache.spark.util.Utils$.$anonfun$getCurrentUserName$1(Utils.scala:2416)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.util.Utils$.getCurrentUserName(Utils.scala:2416)
	at org.apache.spark.SparkContext.<init>(SparkContext.scala:329)
	at org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
	at java.base/jdk.internal.reflect.DirectConstructorHandleAccessor.newInstance(DirectConstructorHandleAccessor.java:62)
	at java.base/java.lang.reflect.Constructor.newInstanceWithCaller(Constructor.java:499)
	at java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:483)
	at py4j.reflection.Metho